In [9]:

import os
from dotenv import load_dotenv
from libs.community.langchain_community.tools.azure_ai_services.azure_translate import AzureTranslateTool
from langchain_core.prompts import PromptTemplate
load_dotenv()  

ModuleNotFoundError: No module named 'libs.community.langchain_community.tools.azure_ai_services.azure_translate'

In [3]:
translate_key = os.getenv('AZURE_OPENAI_TRANSLATE_API_KEY')
translate_endpoint = os.getenv('AZURE_OPENAI_TRANSLATE_ENDPOINT')

assert translate_key is not None, "AZURE_OPENAI_TRANSLATE_API_KEY is not set."
assert translate_endpoint is not None, "AZURE_OPENAI_TRANSLATE_ENDPOINT is not set."

In [4]:
translator = AzureTranslateTool(translate_key=translate_key, translate_endpoint=translate_endpoint)

NameError: name 'AzureTranslateTool' is not defined

In [5]:

prompt = PromptTemplate(
    input_variables=["input"],
    template="You are a translation agent. Use the translation tool to translate the following input: {input}"
)

NameError: name 'PromptTemplate' is not defined

In [6]:
def translate_text(input_text, target_language='es'):
    try:
        # Run the translation
        translated_text = translator._run(input_text, target_language)
        
        # Format the result in a dictionary
        result = {
            'input': input_text,
            'output': f"The text '{input_text}' was translated to '{translated_text}' in {target_language}."
        }
        return result
    except Exception as e:
        return {'error': str(e)}

In [7]:
# Sample text to translate
sample_text = "This is an example of the translation tool made by Capstone Project Team 3"
# Target language
target_language = 'fr' 
# Perform translation using the defined function
translated_result = translate_text(sample_text, target_language)
print(f"Original Text: {translated_result['input']}")
print(f"Translated to {target_language}: {translated_result['output']}")

KeyError: 'input'